In [1]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import numpy as np
import copy
import hashlib
import json
import warnings
import calendar
import datetime
warnings.filterwarnings('ignore')
import pycountry_convert as cc

%matplotlib inline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler


from sklearn.metrics import accuracy_score, precision_score, \
                            recall_score, f1_score, \
                            roc_auc_score, roc_curve, \
                            confusion_matrix

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    GradientBoostingClassifier,
    GradientBoostingRegressor,
)
from sklearn.metrics import mean_squared_error
from sklearn.tree import (
    DecisionTreeClassifier,
    DecisionTreeRegressor,
)
from math import log2

from utils.utils import process_categorical_features

#from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv(os.path.join('data', 'train.csv')) 
#df = pd.read_csv(os.path.join('data', 'test.csv')) 
#df.isnull().sum()

In [11]:
def encode_binary_field(f):

    f_e = f.copy()
    
    ## create a dictionary mapping the current values to int values
    enconding_map = {'very low' : 0, 'low':1, 'medium':2, 'high':3,'very high':4}
    ## change target using the mapping
    f_e = f_e.map(enconding_map)
   
    
    return f_e

def clean_data (df):
    df1 = df.copy()
    
    df1['profit'] = abs(df1.exchange_b_price - df1.exchange_a_price)
    df1['anonymousL'] = df1.anonymous.str.replace('\d','')
    df1['anonymousN'] =df1.anonymous.str.replace('[A-Z]*','') 
    df1['anonymousN']=df1['anonymousN'].astype('int64')
    df1['anonymousN']=df1['anonymousN'].astype('string')
    
    #df_ = df1[['user_balance_usd', 'user_balance_coin_a', 'user_historical_transactions_10s', 'user_historical_transactions_30s', 'exchange_a_price', 'exchange_b_price' , 'exchange_a_volatility','exchange_b_volatility','result','profit','anonymousN','anonymousL']].copy()
    #df_ = df1[['user_balance_usd', 'user_balance_coin_a', 'user_historical_transactions_10s', 'user_historical_transactions_30s' , 'exchange_a_volatility','exchange_b_volatility','result','profit']].copy()
    #df_["anonymous"] =df_["anonymous"].astype("category")
    #df_ = df1[['profit', 'user_balance_coin_a', 'user_historical_transactions_30s', 'user_historical_transactions_30s','result']].copy()
    df_ =df1[['user_address','user_historical_transactions_30s','user_historical_transactions_10s','profit','user_balance_coin_a','user_balance_usd','result']]

    df_ = df_.dropna()
    
    #df_ =convert_categorical_features(df_)
    
    #df_=convert(df_)
    
    return df_

In [12]:
df_clean=clean_data(df)
df_clean

,user_address,user_historical_transactions_30s,user_historical_transactions_10s,profit,user_balance_coin_a,user_balance_usd,result
0,0x930B,0,0,0.089979,44.193560,84139.092267,0
1,0x5779,0,0,0.110526,331.501171,34609.449276,0
2,0x5779,0,0,0.052469,1581.902691,30598.783296,0
3,0x5779,0,0,0.111473,1260.406181,20992.680359,0
4,0xD47A,0,0,0.125942,180.090128,120838.946555,0
...,...,...,...,...,...,...,...
338481,0x5779,0,0,0.165826,352.813282,72909.885451,0
338482,0x5779,0,0,0.012315,352.561329,29374.263175,0
338483,0x5AEe,0,0,288.272833,239.577796,50847.577759,0
338484,0x930B,0,0,0.103284,160.222687,121303.381921,0


In [13]:
df_test = pd.read_csv(os.path.join('data', 'test.csv')) 
df_test

,id,user_address,user_balance_usd,user_balance_coin_a,user_historical_transactions_10s,user_historical_transactions_30s,exchange_a_price,exchange_b_price,exchange_a_volatility,exchange_b_volatility,anonymous
0,3988978,0x5779,51391.615731,1060.556537,0,0,35.954570,35.9667,NaN,NaN,F142
1,3988979,0x5779,91986.384059,218.535933,1,4,45.300626,45.0849,very high,0.000781,M1729
2,3989002,0x5779,41759.237681,1285.867013,0,0,37.481412,37.5039,very low,0.000562,F1542
3,3989008,0x5779,48074.340099,0.535869,0,0,40.753761,40.8303,low,0.000372,M1550
4,3989022,0x5AEe,36021.119553,431.542705,0,0,70.976657,282.6852,low,0.000576,A638
...,...,...,...,...,...,...,...,...,...,...,...
112825,5307776,0x5779,80592.884173,0.706920,0,0,39.991680,39.7971,very low,0.000707,F132
112826,5307781,0xD47A,107779.088520,423.156478,0,0,54.280719,54.2535,low,NaN,A16
112827,5307790,0x5779,118584.595142,110.042328,0,0,71.513852,71.4693,low,0.000264,A549
112828,5307800,0xD47A,88914.452535,695.334793,0,0,62.108812,62.1201,low,0.000284,A358


In [14]:
df_test = pd.read_csv(os.path.join('data', 'test.csv')) 

def clean_data_test (df):
    df1 = df.copy()
    
    df1['profit'] = abs(df1.exchange_b_price - df1.exchange_a_price)
    df1['anonymousL'] = df1.anonymous.str.replace('\d','')
    df1['anonymousN'] =df1.anonymous.str.replace('[A-Z]*','') 
    df1['anonymousN']=df1['anonymousN'].astype('int64')
    df1['anonymousN']=df1['anonymousN'].astype('string')
    
    #df_ = df1[['user_balance_usd', 'user_balance_coin_a', 'user_historical_transactions_10s', 'user_historical_transactions_30s', 'exchange_a_price', 'exchange_b_price' , 'exchange_a_volatility','exchange_b_volatility','result','profit','anonymousN','anonymousL']].copy()
    #df_ = df1[['user_balance_usd', 'user_balance_coin_a', 'user_historical_transactions_10s', 'user_historical_transactions_30s' , 'exchange_a_volatility','exchange_b_volatility','result','profit']].copy()
    #df_["anonymous"] =df_["anonymous"].astype("category")
    #df_ = df1[['profit', 'user_balance_coin_a', 'user_historical_transactions_30s', 'user_historical_transactions_30s','result']].copy()
    df_ =df1[['user_historical_transactions_30s','user_historical_transactions_10s','profit','user_balance_coin_a','user_balance_usd']]

    df_.profit=df_.profit.fillna(0)
    
    return df_

df_test_clean= clean_data_test(df_test)
df_test_clean

,user_historical_transactions_30s,user_historical_transactions_10s,profit,user_balance_coin_a,user_balance_usd
0,0,0,0.012130,1060.556537,51391.615731
1,4,1,0.215726,218.535933,91986.384059
2,0,0,0.022488,1285.867013,41759.237681
3,0,0,0.076539,0.535869,48074.340099
4,0,0,211.708543,431.542705,36021.119553
...,...,...,...,...,...
112825,0,0,0.194580,0.706920,80592.884173
112826,0,0,0.027219,423.156478,107779.088520
112827,0,0,0.044552,110.042328,118584.595142
112828,0,0,0.011288,695.334793,88914.452535


In [21]:
def train_test_split_m(df1,test_df):
    
    df= df1.copy()
    X = df.drop('result',axis=1)
    #X= X.drop('anonymous', axis=1)
    X_ = process_categorical_features(X)
    
    y = df['result']
    

    X_train, X_test, y_train, y_test = train_test_split(
        X_, y, test_size=0.20)
    
    pipeline = make_pipeline(
    RandomForestClassifier()
    )
    
    pipeline.fit(X_train, y_train)
    
    
    predicted_proba= pipeline.predict_proba(X_test)
    
    y_score1=predicted_proba[:,1]
    roc_auc = roc_auc_score(y_score=y_score1, y_true= y_test)
   
    #confmat = confusion_matrix(y_true=y_test, y_pred=x1)
    print('The AUROC of our classifier is = %.3f' % roc_auc)
    
    #y_score1.to_csv(r'HopingForTheBest.csv', index = True)

    return df

In [22]:
train_test_split_m(df_clean,df_test_clean)

The AUROC of our classifier is = 0.842


,user_address,user_historical_transactions_30s,user_historical_transactions_10s,profit,user_balance_coin_a,user_balance_usd,result
0,0x930B,0,0,0.089979,44.193560,84139.092267,0
1,0x5779,0,0,0.110526,331.501171,34609.449276,0
2,0x5779,0,0,0.052469,1581.902691,30598.783296,0
3,0x5779,0,0,0.111473,1260.406181,20992.680359,0
4,0xD47A,0,0,0.125942,180.090128,120838.946555,0
...,...,...,...,...,...,...,...
338481,0x5779,0,0,0.165826,352.813282,72909.885451,0
338482,0x5779,0,0,0.012315,352.561329,29374.263175,0
338483,0x5AEe,0,0,288.272833,239.577796,50847.577759,0
338484,0x930B,0,0,0.103284,160.222687,121303.381921,0
